# Notebook responsável por alguns aspectos de modelagem OneToOne / OneToFew Linking (usando MongoEngine)

## Bibliotecas importantes

In [1]:
from mongoengine import *
import pprint
import numpy as np
import pandas as pd
import unidecode

# para geração de dados fake
from faker import Faker


## Classes úteis

In [2]:
class Endereco (Document):
    logradouro = StringField(max_length=200, required=True)
    numero = StringField(max_length=10, required=True)
    bairro = StringField(max_length=50, required=True)
    cidade = StringField(max_length=200, required=True)
    uf = StringField(max_length=50, required=True)
    cep = StringField(max_length=10, required=True)
    
    def __str__(self):
        return f"{self.logradouro}, {self.numero}"
    
    
class Pessoa (Document):
    cpf = StringField(max_length=20, required=True)
    primeiro_nome = StringField(max_length=200, required=True)
    ultimo_nome = StringField(max_length=400, required=True)
    idade = IntField(min_value=0, max_value=120, required=True)
    data_nascimento = StringField(max_length=20, required=True)
    email = EmailField(max_length=50)
    telefones = ListField(StringField(max_length=50), max_length=4)
    formacoes = ListField(StringField(max_length=50), max_length=4)
    
    # Referência (linking) para um objeto da classe pessoa
    # ref.:https://mongoengine-odm.readthedocs.io/apireference.html#documents
    #endereco = ReferenceField(Endereco, reverse_delete_rule=DENY)
    endereco = ReferenceField(Endereco, reverse_delete_rule=DENY)
    
    def __str__(self):
        return f"{self.pk}, {self.cpf}, {self.primeiro_nome}, Endereço: {self.endereco}"


## Funções uteis

In [3]:
def criar_endereco():
    
    logradouro = fake.street_name()
    numero = fake.building_number()
    bairro = fake.bairro()
    cidade = fake.city()
    uf = fake.administrative_unit()
    cep = fake.postcode()

    endereco = Endereco(logradouro=logradouro, numero=numero, bairro=bairro, cidade=cidade, uf=uf, cep=cep)
    return endereco

In [4]:
def criar_pessoa():
    
    cpf = fake.cpf()
    primeiro_nome = fake.first_name()
    ultimo_nome = fake.last_name()
    idade = np.random.randint(18, 50)
    data_nascimento = fake.date()
    email = f'{cpf}@{fake.free_email_domain()}'
    
    # criando os telefones
    telefones = []
    n = np.random.randint(1, 4)
    for i in range(n):
        telefones.append(fake.msisdn())
    
    # criando as formações
    formacoes = []
    n = np.random.randint(1, 4)
    for i in range(n):
        formacoes.append(fake.job())
    
    endereco = criar_endereco()
    
    pessoa = Pessoa(cpf=cpf, primeiro_nome=primeiro_nome, 
                    ultimo_nome=ultimo_nome, idade=idade, 
                    data_nascimento=data_nascimento, 
                    email=email, telefones=telefones, 
                    formacoes=formacoes, endereco=endereco)
    return pessoa

## Configurações

In [5]:
pp = pprint.PrettyPrinter(compact=True)
# gerando dados fictícios em português do Brasil
fake = Faker(['pt_BR'])

## Inserção de vários documentos

### Conectando ao MongoDB

In [6]:
connect('onetoone_linking')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

### Exemplo - Criação de vários documentos compostos

In [7]:
lista_pessoas = []

for i in range(10):
    lista_pessoas.append(criar_pessoa())
    
pp.pprint(lista_pessoas)

[<Pessoa: None, 456.203.198-06, Lívia, Endereço: Viela Ana Aragão, 6>,
 <Pessoa: None, 032.658.491-98, Yuri, Endereço: Condomínio de Viana, 94>,
 <Pessoa: None, 428.530.179-23, Nathan, Endereço: Ladeira Ana Luiza Melo, 60>,
 <Pessoa: None, 583.492.760-29, Otávio, Endereço: Favela de da Costa, 4>,
 <Pessoa: None, 703.269.148-03, Enzo, Endereço: Viaduto Martins, 72>,
 <Pessoa: None, 341.286.059-05, Benjamin, Endereço: Fazenda de Souza, 47>,
 <Pessoa: None, 730.469.815-20, Clarice, Endereço: Estrada da Cruz, 8>,
 <Pessoa: None, 930.651.248-15, Davi Lucas, Endereço: Via de Costa, 38>,
 <Pessoa: None, 451.793.086-66, Danilo, Endereço: Condomínio Mendes, 26>,
 <Pessoa: None, 582.907.346-38, Vicente, Endereço: Trecho Gabriel Barros, 38>]


In [8]:
pp.pprint(lista_pessoas[0].pk)

None


### Inserindo os documentos no banco de dados

In [9]:
# inserindo 1 documento apenas
lista_pessoas[0].cascade_save()
lista_pessoas[0].save()

<Pessoa: 625f4c518aa345c91185abea, 456.203.198-06, Lívia, Endereço: Viela Ana Aragão, 6>

In [10]:
pp.pprint(lista_pessoas[0].pk)

ObjectId('625f4c518aa345c91185abea')


In [11]:
pp.pprint(lista_pessoas[0].endereco.pk)

ObjectId('625f4c518aa345c91185abe9')


In [12]:
# inserindo todos os outros documentos
for p in lista_pessoas[1:]:
    p.cascade_save()
    p.save()

## Recuperando os documentos inseridos

#### Recuperar todos

In [13]:
for p in Pessoa.objects:
    pp.pprint(p.id)

ObjectId('625f4c518aa345c91185abea')
ObjectId('625f4c528aa345c91185abec')
ObjectId('625f4c528aa345c91185abee')
ObjectId('625f4c528aa345c91185abf0')
ObjectId('625f4c528aa345c91185abf2')
ObjectId('625f4c528aa345c91185abf4')
ObjectId('625f4c528aa345c91185abf6')
ObjectId('625f4c528aa345c91185abf8')
ObjectId('625f4c528aa345c91185abfa')
ObjectId('625f4c528aa345c91185abfc')


In [17]:
pessoa = Pessoa.objects.first()
pp.pprint(pessoa)

<Pessoa: 625f4c518aa345c91185abea, 456.203.198-06, Lívia, Endereço: Viela Ana Aragão, 6>


In [18]:
pp.pprint(pessoa.endereco.pk)

ObjectId('625f4c518aa345c91185abe9')


In [19]:
endereco = Endereco.objects(id='625f4c518aa345c91185abe9').first()
endereco.delete()

OperationError: Could not delete document (Pessoa.endereco refers to it)

In [ ]:
Pessoa.objects(id='625f48b63a1f265e4af5816b')